# Part 5
# Creating and Managing Messages
An Assistant represents an entity that can be configured to respond to a user's messages using several parameters like model, instructions, and tools. 

A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain text, image urls, and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.

## Creating an Assistant and a Thread Review
Let's create an Assistant and Thread to get us set up for success with our Messages. 

We begin with an Assistant. 

In [1]:
from openai import OpenAI

# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

# Create an assistant that uses code interpreter.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    instructions="You are a helpful assistant.",
    name="Message Holder",
    metadata={
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_gDnqDWrQ9gkngdusY7ha8VHf', created_at=1715528696, description=None, instructions='You are a helpful assistant.', metadata={'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4-turbo', name='Message Holder', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Message Holder
{'holds_messages': 'True', 'likes_messages': 'True'}


Now we create an empty thread that will hold our messages. Remember to hang on to that thread ID.

In [2]:
from openai import OpenAI

# Initialize a client instance of the OpenAI API
client = OpenAI()

# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread_holding_messages = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread_holding_messages)


Thread(id='thread_SofVSEqy7mXfIeqVNeYjqqiY', created_at=1715528697, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Creating Messages
To create a message we need to determine which thread to add the message to, the role for the message, and the actual message itself. 

### Simple Message
Here is a simple message with some metadata.

In [3]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="What is a penguin?",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message


Message(id='msg_aN61xWgri8HUskPHwc5V09Ty', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715528698, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY')


msg_aN61xWgri8HUskPHwc5V09Ty
[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')]
What is a penguin?
user


### Message with Vision
Now let's create a more complex message that passes in images.

In [4]:
# Create a file in the system by uploading an image, for vision processing purposes.
vision_file = client.files.create(
    file=open("./artifacts/PuppyDog.jpg", "rb"),  # Opens the image file in binary read mode
    purpose="vision"  # Indicates that the file's purpose is for vision-related processing
)

# Create a message that includes both text and images, asking about differences between them.
message_vision = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread to post the message in
    role="user",  # Role of the poster (e.g., 'user')
    content=[  # Content list that includes text and images
        {
            "type": "text",
            "text": "What is the difference between these images?"
        },
        {
            "type": "image_url",
            "image_url": {"url": "https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg"}
        },
        {
            "type": "image_file",
            "image_file": {"file_id": vision_file.id}  # References the uploaded file by its ID
        },
    ],
)

# Print the message object to output its details.
print(message_vision)


Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_file')], created_at=1715528698, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY')


### Message for Tool Use
Finally, let's pass in a file for Code Interpreter to use.

In [5]:
# Create a file object by uploading a data file for use with assistant tools.
data_file = client.files.create(
    file=open("./artifacts/penguins_size.csv", "rb"),  # Open the CSV file in binary read mode
    purpose="assistants"  # Specify the purpose of the file for assistant tool use
)

# Create a message that instructs to create visualizations from the attached data file.
message_code_interpreter = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Create three visualizations based on the data in this file.",  # Content detailing what needs to be done with the data
    attachments=[  # Attachments that include the file and tools for processing
        {
            "file_id": data_file.id,  # The ID of the uploaded data file
            "tools": [{"type": "code_interpreter"}]  # Tools specified to interpret the code
        }
    ]
)

# Print the message object to output its details.
print(message_code_interpreter)


Message(id='msg_1i1crE7cAjSoZ0SGA06HWAP9', assistant_id=None, attachments=[Attachment(file_id='file-2ksPeamuUnRZKhhOZfnka2g2', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715528699, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY')


# Part 6
## Listing Messages
Let's see what messages we have in our thread at this point.

### Start with a simple list
Just list out everything we have so far. Remember that only the first 100 will be listed so if you have more than 100 you will need to use "after" to go to the next page.

In [6]:
from openai import OpenAI

# Initialize the client using the OpenAI library.
client = OpenAI()

# Retrieve a list of messages from a specific thread using its ID.
thread_messages = client.beta.threads.messages.list(thread_holding_messages.id)
print(thread_messages.data)  # Print the raw data of messages
print("\n")  # Print a newline for better readability of output

# Initialize a counter to keep track of the number of messages.
message_count = 0

# Iterate over each message in the list, printing details and counting them.
for message in thread_messages.data:
    print(message.id)  # Print the unique identifier of each message
    print(message.role)  # Print the role of the message sender
    print(message.content)  # Print the content of the message
    print("\n")  # Print a newline for separation between messages
    message_count += 1  # Increment the message count

# Print the total number of messages processed.
print(f"Total number of messages: {message_count}")


[Message(id='msg_1i1crE7cAjSoZ0SGA06HWAP9', assistant_id=None, attachments=[Attachment(file_id='file-2ksPeamuUnRZKhhOZfnka2g2', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715528699, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY'), Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_

### Limiting and Sorting the Messages
Now let's show the two most recent messages using "limit" and "order".

In [7]:
import datetime
import pytz

from openai import OpenAI

# Initialize the client using the OpenAI library.
client = OpenAI()

# Retrieve a list of messages from a specific thread using its ID.
thread_messages = client.beta.threads.messages.list(
    thread_id=thread_holding_messages.id,
    limit=2,  # Limit the number of messages retrieved to 2
    order="desc",  # Order the messages in descending order (most recent first)
)
print(thread_messages.data)  # Print the raw data of messages
print("\n")  # Print a newline for better readability of output

# Initialize a counter to keep track of the number of messages.
message_count = 0

# Iterate over each message in the list, printing details and counting them.
for message in thread_messages.data:
    print(message.id)  # Print the unique identifier of each message
    print(message.role)  # Print the role of the message sender
    print(message.content)  # Print the content of the message

    # grab the created date/time in unix format 
    # and convert it to a human readable format
    unix_time = message.created_at

    # Create a timezone object for Central Time
    central_timezone = pytz.timezone('America/Chicago')

    # Convert Unix time to a datetime object in UTC
    utc_date = datetime.datetime.fromtimestamp(unix_time)

    # Localize the UTC datetime object to Central Time
    central_date = utc_date.replace(tzinfo=pytz.utc).astimezone(central_timezone)

    # Format the datetime object to a readable string
    formatted_date = central_date.strftime('%Y-%m-%d %I:%M:%S %Z')

    print(formatted_date) # Print the formatted date and time

    print("\n")  # Print a newline for separation between messages

    message_count += 1  # Increment the message count

# Print the total number of messages processed.
print(f"Total number of messages: {message_count}")

[Message(id='msg_1i1crE7cAjSoZ0SGA06HWAP9', assistant_id=None, attachments=[Attachment(file_id='file-2ksPeamuUnRZKhhOZfnka2g2', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715528699, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY'), Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_

### Pagination with Messages
Let's see what forward pagination looks like with the "after" method.

In [8]:
# Import the OpenAI library
from openai import OpenAI

# Create an OpenAI client
client = OpenAI()

# Initialize the 'after_id' variable to track pagination
after_id = None

# Loop to fetch all pages of messages until no more data is available
while True:
    # Retrieve a list of up to 2 messages, sorted from newest to oldest,
    # starting after the previously retrieved message (if any)
    thread_messages = client.beta.threads.messages.list(
        thread_id=thread_holding_messages.id,
        limit=2,  # Limit the number of messages retrieved to 2
        order="desc",  # Order the messages in descending order (most recent first)
        after=after_id  # Include the after_id to fetch messages after the last retrieved ID
    )

    # Check if the retrieved data is empty, indicating no more messages to fetch
    if not thread_messages.data:
        break

    # Process each message in the current page
    for message in thread_messages.data:
        print(message.id)  
        print(message.role)  
        print(message.content)
        print("\n")  

    # Update 'after_id' to the ID of the last assistant in the current page
    # This will be used to fetch the next page in the subsequent iteration
    after_id = thread_messages.data[-1].id

    # Print a marker to indicate the end of the current page
    print("=== END OF PAGE ===\n")


msg_1i1crE7cAjSoZ0SGA06HWAP9
user
[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')]


msg_xZ5K28yfSPCv7smsfzhVxxHk
user
[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_file')]


=== END OF PAGE ===

msg_aN61xWgri8HUskPHwc5V09Ty
user
[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')]


=== END OF PAGE ===



Now let's go backward using the "before" method to see pages.

In [11]:
# Retrieve a list of up to 2 messages, sorted from oldest to newest,
# starting before the first message in the last retrieved page (if any)
thread_messages = client.beta.threads.messages.list(
    thread_id=thread_holding_messages.id,
    limit=2,  # Limit the number of messages retrieved to 2
    order="desc",  # Order the messages in descending order (most recent first)
    before="msg_aN61xWgri8HUskPHwc5V09Ty"  # Include the before_id to fetch messages before the first ID in the last page
)

print(thread_messages.data)  # Print the raw data of messages
print("\n")  # Print a newline for better readability of output

# Process each message in the current page
for message in thread_messages.data:
    print(message.id)  
    print(message.role)  
    print(message.content)
    print("\n")  

[Message(id='msg_1i1crE7cAjSoZ0SGA06HWAP9', assistant_id=None, attachments=[Attachment(file_id='file-2ksPeamuUnRZKhhOZfnka2g2', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715528699, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY'), Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_

## Retrieving a Message
We can always fetch a message if we need to from the thread.

In [12]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# Retrieve a specific message by its ID from a given thread.
fetched_message = client.beta.threads.messages.retrieve(
    thread_id=thread_holding_messages.id,  # ID of the thread containing the message
    message_id=message.id,  # ID of the message to retrieve
)

# Print the retrieved message object to see its details.
print(fetched_message)



Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_file')], created_at=1715528698, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY')


## Modify Messsages
Making changes to messages is easy. Just call the update method.

In [13]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# show the before data
print(f"Before Update: {message}")
print("\n")

# Update a specific message by its ID within a given thread.
modified_message = client.beta.threads.messages.update(
    thread_id=thread_holding_messages.id,  # ID of the thread containing the message
    message_id=message.id,  # ID of the message to update
    metadata={  # Metadata to add or update for the message
        "modified": "true",
        "user": "abc123",
    },
)

# Print the modified message object to view its updated details.
print(modified_message)



Before Update: Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', detail='auto'), type='image_url'), ImageFileContentBlock(image_file=ImageFile(file_id='file-oa0ood9s42HKr7ktEQrFyBvN', detail='auto'), type='image_file')], created_at=1715528698, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_SofVSEqy7mXfIeqVNeYjqqiY')


Message(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', assistant_id=None, attachments=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageURLContentBlock(image_url=ImageURL(url='https://en.wikipedia.org/wiki/File:Cat_August_2010

## Delete Messages
Finally, let's clean up after ourselves and delete any messages we don't need any more.

In [14]:
# You may need to upgrade to the latest version of the OpenAI library to use the delete method.
# If you encounter an error, consider upgrading the library using 'pip install openai --upgrade'
# !pip install openai --upgrade

In [15]:
from openai import OpenAI


# Initialize the OpenAI client.
client = OpenAI()

try:
    # Delete a specific message by its ID within a given thread.
    deleted_message = client.beta.threads.messages.delete(
        thread_id=thread_holding_messages.id,
        message_id=message.id
    )

    # Print the result of the delete operation to verify its success.
    print("Message deleted successfully:", deleted_message)

except NotFoundError as e:
    # Handle the case where the message does not exist
    print("The message could not be found and thus not deleted. Error:", str(e))



Message deleted successfully: MessageDeleted(id='msg_xZ5K28yfSPCv7smsfzhVxxHk', deleted=True, object='thread.message.deleted')


### Cleanup Code
Uncomment the code below to blow away all your messages in our thread if you want to play around with them some more

In [16]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# Assume 'thread_id' is the ID of the thread from which you want to delete all messages
thread_id = thread_holding_messages.id

# First, retrieve all messages from the thread
try:
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    message_ids = [message.id for message in messages]
except Exception as e:
    print(f"Failed to retrieve messages: {e}")
    messages = []

# Loop through each message ID and attempt deletion
# for message_id in message_ids:
#     try:
#         # Attempt to fetch message to check existence
#         message = client.beta.threads.messages.retrieve(message_id=message_id, thread_id=thread_id)
#         # If message exists, proceed with deletion
#         deleted_message = client.beta.threads.messages.delete(message_id=message_id, thread_id=thread_id)
#         print(f"Deleted message ID {message_id}: {deleted_message}")
#     except Exception as e:
#         if "NotFoundError" in str(e):
#             print(f"Message ID {message_id} not found or already deleted.")
#         else:
#             print(f"Failed to delete message ID {message_id}: {e}")
